# PhoBERT NER - Training Model

Notebook này fine-tune PhoBERT cho task Named Entity Recognition (NER) trên dữ liệu y tế tiếng Việt.

## Model
- Pretrained: `vinai/phobert-base` (135M parameters)
- Task: Token Classification (NER)
- Labels: B-SYMPTOM, I-SYMPTOM, B-DISEASE, I-DISEASE, O

In [ ]:
import json
import numpy as np
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  Chạy trên CPU (sẽ chậm hơn)")

e:\CHATBOT MODEL\training\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ PyTorch version: 2.9.1+cpu
✓ CUDA available: False
⚠️  Chạy trên CPU (sẽ chậm hơn)


## 1. Load dữ liệu

In [ ]:
# Đọc training data
with open("../../data/processed/train_phobert.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Đọc validation data
with open("../../data/processed/val_phobert.json", "r", encoding="utf-8") as f:
    val_data = json.load(f)

print(f"✓ Training: {len(train_data)} câu")
print(f"✓ Validation: {len(val_data)} câu")

print("\nVí dụ:")
print(train_data[0])

✓ Training: 320 câu
✓ Validation: 80 câu

Ví dụ:
{'tokens': ['Viêm', 'tai', 'giữa'], 'ner_tags': ['B-DISEASE', 'I-DISEASE', 'I-DISEASE']}


## 2. Tạo Label Mapping

In [ ]:
# Tự động tìm tất cả labels từ data
all_labels = set()
for item in train_data + val_data:
    all_labels.update(item['ner_tags'])

# Sắp xếp labels (O luôn ở đầu)
label_list = sorted(list(all_labels))
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list

# Tạo mapping
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

print(f"✓ Số lượng labels: {len(label_list)}")
print(f"✓ Labels: {label_list}")
print(f"\nLabel mapping:")
for label, idx in label2id.items():
    print(f"  {label} → {idx}")

✓ Số lượng labels: 5
✓ Labels: ['O', 'B-DISEASE', 'B-SYMPTOM', 'I-DISEASE', 'I-SYMPTOM']

Label mapping:
  O → 0
  B-DISEASE → 1
  B-SYMPTOM → 2
  I-DISEASE → 3
  I-SYMPTOM → 4


## 3. Chuyển đổi sang Hugging Face Dataset

In [ ]:
def convert_tags_to_ids(data, label2id):
    """Chuyển text labels thành IDs"""
    converted = []
    for item in data:
        tag_ids = [label2id[tag] for tag in item['ner_tags']]
        converted.append({
            'tokens': item['tokens'],
            'ner_tags': tag_ids
        })
    return converted

train_data_converted = convert_tags_to_ids(train_data, label2id)
val_data_converted = convert_tags_to_ids(val_data, label2id)

# Tạo Dataset objects
train_dataset = Dataset.from_dict({
    'tokens': [item['tokens'] for item in train_data_converted],
    'ner_tags': [item['ner_tags'] for item in train_data_converted]
})

val_dataset = Dataset.from_dict({
    'tokens': [item['tokens'] for item in val_data_converted],
    'ner_tags': [item['ner_tags'] for item in val_data_converted]
})

print("✓ Đã tạo Hugging Face Datasets")
print(f"  Train: {len(train_dataset)} samples")
print(f"  Val: {len(val_dataset)} samples")

✓ Đã tạo Hugging Face Datasets
  Train: 320 samples
  Val: 80 samples


## 4. Load PhoBERT Tokenizer & Model

In [ ]:
model_checkpoint = "vinai/phobert-base"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)
print("✓ Đã load tokenizer")

# Load model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)
print("✓ Đã load PhoBERT model")
print(f"  Parameters: {model.num_parameters():,}")

✓ Đã load tokenizer


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Đã load PhoBERT model
  Parameters: 134,411,525


## 5. Tokenize và Align Labels

In [ ]:
def tokenize_and_align_labels(examples):
    """
    Tokenize text và align labels.
    PhoBERT tokenizer không hỗ trợ word_ids(), nên ta phải xử lý thủ công.
    """
    tokenized_inputs = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for tokens, labels in zip(examples["tokens"], examples["ner_tags"]):
        # Tokenize từng token riêng lẻ
        tokenized_tokens = []
        aligned_labels = []
        
        for token, label in zip(tokens, labels):
            # Tokenize token hiện tại
            token_ids = tokenizer.encode(token, add_special_tokens=False)
            tokenized_tokens.extend(token_ids)
            
            # Label cho token đầu tiên, -100 cho các subword
            aligned_labels.append(label)
            aligned_labels.extend([-100] * (len(token_ids) - 1))
        
        # Thêm special tokens [CLS] và [SEP]
        input_ids = [tokenizer.cls_token_id] + tokenized_tokens + [tokenizer.sep_token_id]
        attention_mask = [1] * len(input_ids)
        labels_with_special = [-100] + aligned_labels + [-100]
        
        # Truncate nếu quá dài
        max_length = 256
        if len(input_ids) > max_length:
            input_ids = input_ids[:max_length]
            attention_mask = attention_mask[:max_length]
            labels_with_special = labels_with_special[:max_length]
        
        tokenized_inputs["input_ids"].append(input_ids)
        tokenized_inputs["attention_mask"].append(attention_mask)
        tokenized_inputs["labels"].append(labels_with_special)
    
    return tokenized_inputs

# Tokenize datasets
print("🔄 Tokenizing data...")
tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=val_dataset.column_names)
print("✓ Hoàn thành tokenization")

# Kiểm tra 1 sample
print("\n🔍 Kiểm tra tokenization:")
sample = tokenized_train[0]
print(f"  Input IDs length: {len(sample['input_ids'])}")
print(f"  Labels length: {len(sample['labels'])}")
print(f"  Attention mask length: {len(sample['attention_mask'])}")

🔄 Tokenizing data...


Map: 100%|██████████| 80/80 [00:00<00:00, 1594.55 examples/s]

✓ Hoàn thành tokenization

🔍 Kiểm tra tokenization:
  Input IDs length: 5
  Labels length: 5
  Attention mask length: 5


## 6. Data Collator

In [ ]:
# Data collator tự động padding batch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
print("✓ Đã tạo data collator")

✓ Đã tạo data collator


## 7. Metrics Computation

In [ ]:
def compute_metrics(eval_preds):
    """
    Tính precision, recall, F1 score cho NER task.
    Sử dụng seqeval metrics.
    """
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Chuyển predictions và labels thành text labels
    true_labels = []
    true_predictions = []

    for prediction, label in zip(predictions, labels):
        true_label = []
        true_pred = []
        for pred, lab in zip(prediction, label):
            if lab != -100:  # Bỏ qua padding và subword tokens
                true_label.append(id2label[lab])
                true_pred.append(id2label[pred])
        true_labels.append(true_label)
        true_predictions.append(true_pred)

    # Tính metrics
    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

print("✓ Đã định nghĩa metrics function")

✓ Đã định nghĩa metrics function


## 8. Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="../../models/phobert_ner_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="../models/logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    save_total_limit=2  # Chỉ giữ 2 checkpoints tốt nhất
)

print("✓ Training configuration:")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Weight decay: {training_args.weight_decay}")

✓ Training configuration:
  Learning rate: 2e-05
  Batch size: 8
  Epochs: 5
  Weight decay: 0.01


## 9. Initialize Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✓ Đã khởi tạo Trainer")

✓ Đã khởi tạo Trainer


## 10. Train Model 🚀

In [ ]:
print("\n🚀 Bắt đầu training...\n")
print("=" * 60)

train_result = trainer.train()

print("\n" + "=" * 60)
print("✅ Hoàn thành training!")
print(f"\nTraining metrics:")
print(f"  Train loss: {train_result.training_loss:.4f}")
print(f"  Train runtime: {train_result.metrics['train_runtime']:.2f}s")


🚀 Bắt đầu training...



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,1.117700,0.973632,0.574713,0.602410,0.588235


KeyboardInterrupt: 

## 11. Evaluate Model

In [ ]:
print("\n📊 Đánh giá model trên validation set...\n")

eval_results = trainer.evaluate()

print("\n" + "=" * 60)
print("📈 Validation Results:")
print("=" * 60)
print(f"Precision: {eval_results['eval_precision']:.4f}")
print(f"Recall:    {eval_results['eval_recall']:.4f}")
print(f"F1 Score:  {eval_results['eval_f1']:.4f}")
print(f"Loss:      {eval_results['eval_loss']:.4f}")
print("=" * 60)


📊 Đánh giá model trên validation set...




📈 Validation Results:
Precision: 1.0000
Recall:    1.0000
F1 Score:  1.0000
Loss:      0.0194


## 12. Lưu Model

In [ ]:
# Lưu model và tokenizer
output_dir = "../../models/phobert_ner_model"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ Đã lưu model vào: {output_dir}")

# Lưu label mapping
import json
with open(f"{output_dir}/label_mapping.json", "w", encoding="utf-8") as f:
    json.dump({"label2id": label2id, "id2label": id2label}, f, ensure_ascii=False, indent=2)

print(f"✓ Đã lưu label mapping")

✓ Đã lưu model vào: ../../models/phobert_ner_model
✓ Đã lưu label mapping


## 13. Test Model với Câu Mẫu

In [ ]:
from transformers import pipeline

# Tạo NER pipeline
ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"  # Gộp B- và I- thành 1 entity
)

# Test cases
test_texts = [
    "Tôi bị sốt cao và ho khan",
    "Em bé có triệu chứng đau bụng và buồn nôn",
    "Cảm cúm gây ra các triệu chứng như sốt, ho và mệt mỏi",
    "Tôi bị đau đầu dữ dội và chóng mặt",
    "Bệnh nhân có triệu chứng sổ mũi và nghẹt mũi"
]

print("\n" + "=" * 60)
print("🧪 TEST MODEL VỚI CÁC CÂU MẪU")
print("=" * 60)

for i, text in enumerate(test_texts, 1):
    print(f"\n[{i}] Input: {text}")
    
    entities = ner_pipeline(text)
    
    if entities:
        print("    Entities tìm được:")
        for ent in entities:
            print(f"      • {ent['word']:<20} [{ent['entity_group']}] (score: {ent['score']:.3f})")
    else:
        print("    (Không tìm thấy entity nào)")

print("\n" + "=" * 60)
print("\n✅ HOÀN THÀNH!")
print("\n📝 Bước tiếp theo: Chạy notebook 03_compare_spacy_phobert.ipynb để so sánh với Spacy")

Device set to use cpu



🧪 TEST MODEL VỚI CÁC CÂU MẪU

[1] Input: Tôi bị sốt cao và ho khan
    Entities tìm được:
      • sốt cao              [SYMPTOM] (score: 0.953)
      • ho khan              [SYMPTOM] (score: 0.958)

[2] Input: Em bé có triệu chứng đau bụng và buồn nôn
    Entities tìm được:
      • đau bụng             [SYMPTOM] (score: 0.972)
      • buồn nôn             [SYMPTOM] (score: 0.972)

[3] Input: Cảm cúm gây ra các triệu chứng như sốt, ho và mệt mỏi
    Entities tìm được:
      • Cảm cúm              [DISEASE] (score: 0.909)
      • số@@                 [SYMPTOM] (score: 0.877)
      • ho                   [SYMPTOM] (score: 0.918)
      • mệt mỏi              [SYMPTOM] (score: 0.965)

[4] Input: Tôi bị đau đầu dữ dội và chóng mặt
    Entities tìm được:
      • đau đầu              [SYMPTOM] (score: 0.950)
      • dữ dội               [SYMPTOM] (score: 0.843)
      • chóng mặt            [SYMPTOM] (score: 0.967)

[5] Input: Bệnh nhân có triệu chứng sổ mũi và nghẹt mũi
    Entities tìm được: